In [2]:
%load_ext sql
import psycopg2
import psycopg2.extras



In [3]:
conn = psycopg2.connect(host="postgisdb", port = 5432, database="dsagis", user="docker", password="docker")
conn.set_session(autocommit=True)
cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)


In [4]:
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS TestRectangleShapes;''')
cur.execute('''CREATE TABLE TestRectangleShapes (
    
    id              SERIAL PRIMARY KEY,
    
    slide_id VARCHAR(50),
	x1 INTEGER,
	x2 INTEGER,
	y1 INTEGER,
	y2 INTEGER,
	shapeName VARCHAR(30),
    shapeLabel VARCHAR(30),
	shapeLocation GEOMETRY
);''')

In [5]:
# insert_example = ''' insert into TestRectangleShapes (slide_id,x1,x2,y1,y2,shapeName, shapeLabel)
#      VALUES ('slideUno', 20,30,40,50, 'Donkey','iNFTtbd');'''

# cur.execute(insert_example)


In [9]:
geopandas.__version__

NameError: name 'geopandas' is not defined

In [ ]:
import random 
### Let's generate 500 random shapes...
numShapesToGen = 1000
max_X = 10000
max_Y = 10000
boxSize_X = 40
boxSixe_Y = 50

colList = '''insert into TestRectangleShapes (slide_id,x1,x2,y1,y2,shapeName, shapeLabel,shapeLocation)'''
colList += " VALUES "

for i in range(numShapesToGen):
    x1 = random.randrange( 0 ,max_X)
    y1 = random.randrange( 0, max_Y)
    
    x2 = x1 + boxSize_X
    y2 = y1 + boxSixe_Y

    ##Insert statement values would be..
    valList= f"('slideUno',{x1},{x2},{y1},{y2},'box','NFT', ST_MakeEnvelope({x1},{y1},{x2},{y2},4326))"
    colList+= valList + ", "
colList = colList[:-2]    
cur.execute(colList) 

In [10]:
cur.execute('create schema dsa')
cur.fetchall()


ProgrammingError: no results to fetch

In [ ]:
# cur.execute('select * from TestRectangleShapes where shapeLocation is NULL')
# d= cur.fetchall()

In [ ]:
### Let's find if I created any overlapping polygoms

pgis_overlappolygons = '''
select *
  from TestRectangleShapes as t1, TestRectangleshapes as t2
  where ST_Overlaps(t1.shapeLocation, t2.shapeLocation) = true
    and t1.id < t2.id'''

In [ ]:
### Let's find if I created any overlapping polygoms

pgis_overlappolygons = '''
select t1.id as t1_id, t1.x1 as t1_x1 , t1.y1 as t1_y1, t1.x2 as t1_x2, t1.y2 as t1_y2, 
  t2.id, t2.x1,t2.y1,t2.x2,t2.y2 
  from TestRectangleShapes as t1, TestRectangleshapes as t2
  where ST_Overlaps(t1.shapeLocation, t2.shapeLocation) = true
    and t1.id < t2.id and t1.id=2'''

In [ ]:
cur.execute(pgis_overlappolygons)
overlapPolyList = cur.fetchall()

In [ ]:
import pandas as pd

pd.DataFrame(overlapPolyList)